# Day 19: Scanner
The input for this problem is located at https://adventofcode.com/2021/day/19/input

In [1]:
import collections
import dataclasses
import itertools

%load_ext numpy_html

We will start with a `Vector3` class to handle vectors

In [2]:
@dataclasses.dataclass(frozen=True)
class Vector3:
    x: int
    y: int
    z: int

    def __add__(self, other):
        if not isinstance(other, Vector3):
            return NotImplemented

        return Vector3(x=self.x + other.x, y=self.y + other.y, z=self.z + other.z)

    def __sub__(self, other):
        if not isinstance(other, Vector3):
            return NotImplemented

        return self + (-other)

    def __neg__(self):
        return Vector3(x=-self.x, y=-self.y, z=-self.z)

    def __mul__(self, other):
        if isinstance(other, Vector3):
            return self.x * other.x + self.y * other.y + self.z * other.z
        if isinstance(other, int):
            return Vector3(self.x * other, self.y * other, self.z * other)
        return NotImplemented

    def cross(self, other):
        return Vector3(
            x=(self.y * other.z - self.z * other.y),
            y=(self.z * other.x - self.x * other.z),
            z=(self.x * other.y - self.y * other.x),
        )

    @property
    def mag2(self) -> float:
        return self.x ** 2 + self.y ** 2 + self.z ** 2

In [3]:
@dataclasses.dataclass(frozen=True)
class Rotation:
    u: Vector3
    v: Vector3
    w: Vector3

    def __matmul__(self, other):
        if not isinstance(other, Vector3):
            return NotImplemented

        return Vector3(self.u * other, self.v * other, self.w * other)

In [24]:
@dataclasses.dataclass(frozen=True)
class Transform:
    translation: Vector3
    rotation: Rotation

    def __add__(self, other):
        if not isinstance(other, Vector3):
            return NotImplemented

        return Transform(self.translation + other, self.rotation)

    def __sub__(self, other):
        if not isinstance(other, Vector3):
            return NotImplemented

        return self + (-other)

    def __matmul__(self, other):
        if not isinstance(other, Vector3):
            return NotImplemented

        return self.translation + self.rotation @ other

We can load the coordinates from our input and parse them into vectors

In [25]:
coordinates = []

with open("input.txt") as f:
    for line in f:
        if line.startswith("---"):
            coordinates.append([])
        elif line.strip():
            x, y, z = map(int, line.split(","))
            coordinates[-1].append(Vector3(x, y, z))

Now we want to be able to use a heuristic to find overlapping regions, and determine which points are common between them. We will create a `Graph` class that uses the transformation-invariant distance between nodes.

In [26]:
@dataclasses.dataclass(frozen=True)
class Graph:
    edges_by_separation: dict[float, list[tuple[int, int]]]
    separations_by_node: list[set[float]]

We build the graph by using a euclidean distance separation metric

In [27]:
def build_graph(points):
    edges_by_separation = collections.defaultdict(list)
    separations_by_node = collections.defaultdict(set)

    n_points = len(points)
    for i, j in itertools.combinations(range(n_points), 2):
        separation = (points[i] - points[j]).mag2
        edges_by_separation[separation].append((i, j))
        separations_by_node[i].add(separation)
        separations_by_node[j].add(separation)

    return Graph(
        edges_by_separation,
        [separations_by_node[i] for i in range(n_points)],
    )

Now, when we ultimately compute two local vectors that likely correspond to the same edge, we need to determine the valid transformation. We can do this most simply (and stupidly!) by generating the possible transforms.

In [28]:
canon = (1, 0, 0), (0, 1, 0), (0, 0, 1)

In [29]:
def orientation_transforms():
    for (s1, s2), (p, q) in itertools.product(
        itertools.product([1, -1], repeat=2), itertools.permutations(canon, 2)
    ):
        # for a, b in itertools.
        n1, n2 = Vector3(*p) * s1, Vector3(*q) * s2
        if n1 * n2:
            continue

        n3 = n1.cross(n2)
        yield Rotation(n1, n2, n3)

Now we can use these generated orientation matrices to compute the transform between the reference and the candidate

In [43]:
def compute_possible_transform(reference, candidate, common_separations):
    for d in common_separations:
        for (i, j), (k, l) in itertools.product(
            reference.graph.edges_by_separation[d],
            candidate.graph.edges_by_separation[d],
        ):
            i_and_l = len(
                reference.graph.separations_by_node[i]
                & candidate.graph.separations_by_node[l]
            )
            j_and_l = len(
                reference.graph.separations_by_node[j]
                & candidate.graph.separations_by_node[l]
            )

            # We will take our relative vectors using the relative
            # origins of u, and p.
            # This means we want q and v to be the "same" node
            # We can do this by finding the node in {u, v}
            # which has the most similar separations.
            # Here, we just switch the indices to guarantee this
            if j_and_l < i_and_l:
                i, j = j, i

            u, v = reference.points[i], reference.points[j]
            p, q = candidate.points[k], candidate.points[l]

            v_u = v - u
            q_p = q - p

            for r_qp_vu in orientation_transforms():
                if (r_qp_vu @ q_p) != v_u:
                    continue

                # Create new transform including translation
                dr = v - r_qp_vu @ q
                yield Transform(dr, r_qp_vu)

In [31]:
@dataclasses.dataclass(frozen=True)
class UnknownRegion:
    graph: Graph
    points: list[Vector3]

In [32]:
@dataclasses.dataclass(frozen=True)
class KnownRegion:
    graph: Graph
    points: list[Vector3]
    origin: Vector3

In [44]:
def update_knowledge(known, unknown):
    candidates = (
        (
            reference,
            candidate,
            candidate.graph.edges_by_separation.keys()
            & reference.graph.edges_by_separation.keys(),
        )
        for reference, candidate in itertools.product(known, unknown)
    )

    for reference, candidate, common_separations in sorted(
        candidates, key=lambda x: len(x[2]), reverse=True
    ):
        # If we have at least 12 common beacons in each set,
        # then common to both beacons we should have nCk(12, 2) distances
        # corresponding to the number inter-beacon distances for each common
        # beacon (2 beacons form an edge, 12 beacons)
        # However, without ensuring uniqueness of our distance computation,
        # we may see fewer separations
        # if len(common_separations) < math.comb(12, 2):
        #     continue

        for transform in compute_possible_transform(
            reference, candidate, common_separations
        ):
            new_points = [transform @ p for p in candidate.points]

            if len(frozenset(reference.points) & frozenset(new_points)) < 12:
                continue

            region = KnownRegion(candidate.graph, new_points, transform.translation)
            return known + [region], [n for n in unknown if n != candidate]
    raise ValueError

In [45]:
known = [KnownRegion(build_graph(coordinates[0]), coordinates[0], Vector3(0, 0, 0))]
unknown = [UnknownRegion(build_graph(p), p) for p in coordinates[1:]]

i = 0
while unknown:
    known, unknown = update_knowledge(known, unknown)
    i += 1

## Part 1

In [46]:
points = set()
for n in known:
    points.update(n.points)

In [47]:
len(points)

79

## Part 2

In [48]:
def manhattan(this, that):
    return abs(this.x - that.x) + abs(this.y - that.y) + abs(this.z - that.z)

In [40]:
separations_manhattan = [
    manhattan(p, q) for p, q in itertools.combinations([n.origin for n in known], 2)
]

In [41]:
max(separations_manhattan)

3621